In [ ]:
import json, os
import cv2
import torch
from multiprocessing import Pool, Manager
from research.utils.data_access_utils import S3AccessUtils, RDSAccessUtils
from research.weight_estimation.akpd import AKPD
from research.weight_estimation.biomass_estimator import NormalizeCentered2D, NormalizedStabilityTransform, ToTensor, Network
from research.weight_estimation.weight_estimator import WeightEstimator
from research.weight_estimation.akpd_scorer import generate_confidence_score
from keras.models import load_model
from research.weight_estimation.optics import pixel2world
import boto3
import pandas as pd
import numpy as np
import plotly.express as px
import time

from matplotlib import pyplot as plt
from PIL import Image
from copy import copy

from collections import defaultdict
import datetime as dt
import json
import numpy as np
from sklearn.linear_model import LinearRegression
from collections import defaultdict



In [ ]:
df = pd.concat([
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-05,2019-06-12).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-12,2019-06-19).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-19,2019-06-26).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-06-26,2019-07-03).csv'),
    pd.read_csv('/root/data/alok/biomass_estimation/playground/output-pen=5/biomass_output,pen=5,range=(2019-07-03,2019-07-04).csv')
])

df = df.sort_values('captured_at')
df['estimated_weight_g'] = df.weight
df = df[df.akpd_score > 0.9].copy(deep=True)
df.index = pd.to_datetime(df.captured_at)
df['hour'] = df.index.hour


In [ ]:
camera_metadata = {"baseline": 0.10079791852561114, "focalLength": 0.013842509663066934, "pixelCountWidth": 4096, "focalLengthPixel": 4012.3216414686767, "imageSensorWidth": 0.01412, "pixelCountHeight": 3000, "imageSensorHeight": 0.01035}

In [ ]:
# instantiate weight estimator class

model_f = '/root/data/alok/biomass_estimation/playground/nn_epoch_253.pb'
weight_estimator = WeightEstimator(model_f)

# generate sample predictions
weights = []
for idx, row in df.iterrows():
    keypoints = json.loads(row.annotation)
    weight_prediction = weight_estimator.predict(keypoints, camera_metadata)
    weights.append(weight_prediction)
    if len(weights) % 1000 == 0:
        print(len(weights))


In [ ]:
np.mean(weights)

In [ ]:
df.weight.mean()

In [ ]:
def get_world_keypoints(x):
    return pixel2world(x['leftCrop'], 
                       x['rightCrop'], 
                       camera_metadata)

def compute_yaw_angle(wkp):
    v = wkp['UPPER_LIP'] - wkp['TAIL_NOTCH']
    yaw_angle = np.arctan(v[1] / v[0]) * 180.0 / np.pi
    return yaw_angle

df['world_keypoints'] = df.annotation.apply(lambda x: get_world_keypoints(json.loads(x)))
df['median_depth'] = df.world_keypoints.apply(lambda x: np.median([wkp[1] for wkp in x.values()]))
df['yaw_angle'] = df.world_keypoints.apply(lambda x: compute_yaw_angle(x))

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.yaw_angle, bins=100)
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.median_depth, bins=100)
plt.grid()
plt.show()

In [ ]:
df['difference'] = np.array(weights) - df.weight

In [ ]:
depth_values = np.arange(0.5, 2.0, 0.1)
for idx in range(len(depth_values) - 1):
    low, high = depth_values[idx], depth_values[idx + 1]
    mean_difference = df[(df.median_depth > low) & (df.median_depth < high)].difference.median()
    print('Mean difference at depth range {}-{}: {}'.format(round(low, 1), round(high, 1), 
                                                            round(mean_difference, 2)))

In [ ]:
depth_values = np.arange(0.5, 2.0, 0.1)
for idx in range(len(depth_values) - 1):
    low, high = depth_values[idx], depth_values[idx + 1]
    std_difference = df[(df.median_depth > low) & (df.median_depth < high)].difference.std()
    print('Std difference at depth range {}-{}: {}'.format(round(low, 1), round(high, 1), 
                                                            round(std_difference, 2)))

In [ ]:
yaw_values = np.arange(-60, 60, 10)
low_depth = 1.6
for idx in range(len(yaw_values) - 1):
    low, high = yaw_values[idx], yaw_values[idx + 1]
    mean_difference = df[(df.median_depth > low_depth) & (df.median_depth < low_depth + 0.1) & \
                         (df.yaw_angle > low) & (df.yaw_angle < high)].difference.mean()
    print('Mean difference at yaw range {}-{}: {}'.format(round(low, 1), round(high, 1), 
                                                            round(mean_difference, 2)))


In [ ]:
plt.figure(figsize=(20, 10))
plt.hist(df.difference, bins=100)
plt.grid()
plt.show()

In [ ]:
def display_crops(left_image, right_image, ann, overlay_keypoints=True, show_labels=False):
    fig, axes = plt.subplots(2, 1, figsize=(20, 20))
    axes[0].imshow(left_image)
    axes[1].imshow(right_image)
    
    left_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['leftCrop']}
    right_keypoints = {item['keypointType']: [item['xCrop'], item['yCrop']] for item in ann['rightCrop']}
    if overlay_keypoints:
        for bp, kp in left_keypoints.items():
            axes[0].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[0].annotate(bp, (kp[0], kp[1]), color='red')
        for bp, kp in right_keypoints.items():
            axes[1].scatter([kp[0]], [kp[1]], color='red', s=1)
            if show_labels:
                axes[1].annotate(bp, (kp[0], kp[1]), color='red')
    plt.show()

In [ ]:
s3_access_utils = S3AccessUtils('/root/data')

In [ ]:
idx = 6
mask = (df.median_depth > 1.8) & (df.median_depth < 1.9)
tdf = df[mask].copy(deep=True)
left_crop_url = tdf.left_crop_url.iloc[idx]
right_crop_url = tdf.right_crop_url.iloc[idx]
print(tdf.difference.iloc[idx])
left_crop_f, _, left_crop_key = s3_access_utils.download_from_url(left_crop_url)
right_crop_f, _, right_crop_key = s3_access_utils.download_from_url(right_crop_url)
left_crop = Image.open(left_crop_f)
right_crop = Image.open(right_crop_f)

left_raw_image_key = os.path.join(os.path.dirname(left_crop_key), 'left_frame.resize_512_512.jpg')
right_raw_image_key = os.path.join(os.path.dirname(right_crop_key), 'right_frame.resize_512_512.jpg')
left_raw_image_f = s3_access_utils.download_from_s3('aquabyte-frames-resized-inbound', left_raw_image_key)
right_raw_image_f = s3_access_utils.download_from_s3('aquabyte-frames-resized-inbound', right_raw_image_key)

left_image = Image.open(left_crop_f)
right_image = Image.open(right_crop_f)
ann = json.loads(tdf.annotation.iloc[idx])
display_crops(left_crop, right_crop, ann)

In [ ]:
Image.open(left_raw_image_f)

In [ ]:
Image.open(right_raw_image_f)